# AWS SageMaker Resource Cleanup

**Project:** AAI-540 Machine Learning Operations - Final Team Project  
**Objective:** Automated cleanup of ALL SageMaker resources in the account

In [1]:
import warnings
warnings.filterwarnings('ignore')

import boto3
import sagemaker
from datetime import datetime

# Initialize AWS clients
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
bucket = sagemaker_session.default_bucket()
sagemaker_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')
cloudwatch_client = boto3.client('cloudwatch')

print(f"Region: {region}")
print(f"Bucket: {bucket}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Region: us-east-1
Bucket: sagemaker-us-east-1-864106638709


In [2]:
def cleanup_all_sagemaker_resources(delete_s3=False, s3_prefixes=None):
    """Delete all SageMaker resources in the account."""
    stats = {
        'endpoints': 0,
        'endpoint_configs': 0,
        'models': 0,
        'monitoring_schedules': 0,
        'processing_jobs': 0,
        'model_package_groups': 0,
        'cloudwatch_alarms': 0,
        's3_objects': 0
    }
    
    # Delete monitoring schedules
    try:
        response = sagemaker_client.list_monitoring_schedules(MaxResults=100)
        for schedule in response.get('MonitoringScheduleSummaries', []):
            try:
                schedule_name = schedule['MonitoringScheduleName']
                if schedule['MonitoringScheduleStatus'] in ['Scheduled', 'Running']:
                    sagemaker_client.stop_monitoring_schedule(MonitoringScheduleName=schedule_name)
                sagemaker_client.delete_monitoring_schedule(MonitoringScheduleName=schedule_name)
                stats['monitoring_schedules'] += 1
            except:
                pass
    except:
        pass
    
    # Stop running processing jobs
    try:
        response = sagemaker_client.list_processing_jobs(StatusEquals='InProgress', MaxResults=100)
        for job in response.get('ProcessingJobSummaries', []):
            try:
                sagemaker_client.stop_processing_job(ProcessingJobName=job['ProcessingJobName'])
                stats['processing_jobs'] += 1
            except:
                pass
    except:
        pass
    
    # Delete CloudWatch alarms
    try:
        response = cloudwatch_client.describe_alarms(AlarmNamePrefix='sagemaker-', MaxRecords=100)
        alarm_names = [alarm['AlarmName'] for alarm in response.get('MetricAlarms', [])]
        if alarm_names:
            cloudwatch_client.delete_alarms(AlarmNames=alarm_names)
            stats['cloudwatch_alarms'] = len(alarm_names)
    except:
        pass
    
    # Delete endpoints
    try:
        response = sagemaker_client.list_endpoints(MaxResults=100)
        for ep in response.get('Endpoints', []):
            try:
                sagemaker_client.delete_endpoint(EndpointName=ep['EndpointName'])
                stats['endpoints'] += 1
            except:
                pass
    except:
        pass
    
    # Delete endpoint configurations
    try:
        response = sagemaker_client.list_endpoint_configs(MaxResults=100)
        for config in response.get('EndpointConfigs', []):
            try:
                sagemaker_client.delete_endpoint_config(EndpointConfigName=config['EndpointConfigName'])
                stats['endpoint_configs'] += 1
            except:
                pass
    except:
        pass
    
    # Delete models
    try:
        response = sagemaker_client.list_models(MaxResults=100)
        for model in response.get('Models', []):
            try:
                sagemaker_client.delete_model(ModelName=model['ModelName'])
                stats['models'] += 1
            except:
                pass
    except:
        pass
    
    # Delete model package groups
    try:
        response = sagemaker_client.list_model_package_groups(MaxResults=100)
        for mpg in response.get('ModelPackageGroupSummaryList', []):
            try:
                group_name = mpg['ModelPackageGroupName']
                pkg_response = sagemaker_client.list_model_packages(ModelPackageGroupName=group_name)
                for pkg in pkg_response.get('ModelPackageSummaryList', []):
                    try:
                        sagemaker_client.delete_model_package(ModelPackageName=pkg['ModelPackageArn'])
                    except:
                        pass
                sagemaker_client.delete_model_package_group(ModelPackageGroupName=group_name)
                stats['model_package_groups'] += 1
            except:
                pass
    except:
        pass
    
    # Delete S3 artifacts
    if delete_s3:
        if s3_prefixes is None:
            s3_prefixes = ['cms-open-payments-light', 'cms-anomaly-detection']
        try:
            for prefix in s3_prefixes:
                try:
                    paginator = s3_client.get_paginator('list_objects_v2')
                    pages = paginator.paginate(Bucket=bucket, Prefix=prefix)
                    for page in pages:
                        if 'Contents' in page:
                            for obj in page['Contents']:
                                s3_client.delete_object(Bucket=bucket, Key=obj['Key'])
                                stats['s3_objects'] += 1
                except:
                    pass
        except:
            pass
    
    # Summary
    print("\nCleanup Summary:")
    print(f"Monitoring Schedules: {stats['monitoring_schedules']}")
    print(f"Processing Jobs: {stats['processing_jobs']}")
    print(f"CloudWatch Alarms: {stats['cloudwatch_alarms']}")
    print(f"Endpoints: {stats['endpoints']}")
    print(f"Endpoint Configs: {stats['endpoint_configs']}")
    print(f"Models: {stats['models']}")
    print(f"Model Package Groups: {stats['model_package_groups']}")
    if delete_s3:
        print(f"  S3 Objects: {stats['s3_objects']}")
    
    return stats

In [3]:
# Execute cleanup - Set delete_s3=True to also delete S3 artifacts
stats = cleanup_all_sagemaker_resources(
    delete_s3=False,  # Change to True to delete S3 data
    s3_prefixes=['cms-open-payments-light', 'cms-anomaly-detection']
)


Cleanup Summary:
Monitoring Schedules: 0
Processing Jobs: 0
CloudWatch Alarms: 0
Endpoints: 13
Endpoint Configs: 14
Models: 13
Model Package Groups: 3
